In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [8]:
df = pd.read_parquet("../data/usa_00001_clean.parquet")

df.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEFIP,STRATA,GQ,PERNUM,...,EMPSTATD,LABFORCE,CLASSWKR,CLASSWKRD,OCC,IND,WKSWORK2,UHRSWORK,FTOTINC,INCWAGE
0,2023,202301,2.0,2.023010e+12,27.0,2.023000e+12,1,10001.0,3,1.0,...,30,1,0,0,0,0,0,0,9999999.0,0.0
1,2023,202301,3.0,2.023010e+12,47.0,2.023000e+12,1,40201.0,4,1.0,...,30,1,0,0,0,0,0,0,9999999.0,0.0
2,2023,202301,5.0,2.023010e+12,57.0,2.023000e+12,1,280201.0,4,1.0,...,30,1,2,22,7810,4971,2,28,9999999.0,2500.0
3,2023,202301,6.0,2.023010e+12,52.0,2.023000e+12,1,120201.0,4,1.0,...,30,1,2,28,4600,7860,1,30,9999999.0,500.0
4,2023,202301,7.0,2.023010e+12,43.0,2.023000e+12,1,180101.0,3,1.0,...,30,1,2,22,8990,3570,0,0,9999999.0,0.0


In [9]:
df_cleaned = df[(df["INCWAGE"] > 0) & df["EMPSTAT"].isin([1, 2])]
df_cleaned = df_cleaned[["AGE", "SEX", "RACE", "MARST", "EDUC", "STATEFIP", "INCWAGE", "CITIZEN"]]
df_cleaned = df_cleaned[df_cleaned["CITIZEN"].isin([2, 3])]
df_cleaned["CITIZEN"] = df_cleaned["CITIZEN"].map({2:1, 3:0})
df_cleaned.head()

,AGE,SEX,RACE,MARST,EDUC,STATEFIP,INCWAGE,CITIZEN
842,49,2,6,6,6,1,7000.0,1
1125,49,2,6,6,6,1,7000.0,1
1875,49,2,6,6,6,1,7000.0,1
2945,53,2,7,1,3,1,37400.0,1
2946,46,1,7,1,0,1,360.0,0


In [10]:
df_cleaned["INCWAGE_LOG"] = np.log(df['INCWAGE'])
categorical_vars = ['SEX', 'RACE', 'MARST', 'EDUC', 'STATEFIP']
for col in categorical_vars:
    df_cleaned[col] = df_cleaned[col].astype('category')
df_cleaned.head()

/Users/brendaz/anaconda3/envs/cs224n/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,AGE,SEX,RACE,MARST,EDUC,STATEFIP,INCWAGE,CITIZEN,INCWAGE_LOG
842,49,2,6,6,6,1,7000.0,1,8.853665
1125,49,2,6,6,6,1,7000.0,1,8.853665
1875,49,2,6,6,6,1,7000.0,1,8.853665
2945,53,2,7,1,3,1,37400.0,1,10.529426
2946,46,1,7,1,0,1,360.0,0,5.886104


In [11]:
df_cleaned.to_csv("../data/cleaned.csv")

Outcome regression suggests a 21% wage premium for naturalized immigrants. Because naturalization is not randomly assigned, this may reflect both causal effects and unobserved selection. Therefore, we next apply causal methods (e.g., IPW, AIPW, overlap weighting) to obtain more credible treatment effects.